<h1>Imports

In [1]:
import pandas as pd
from pyspark.sql import (SparkSession,
                        functions as F,
                        DataFrame,
                        Window)
import matplotlib as plt


<h3>Create spark session

In [ ]:

spark = SparkSession.builder \
    .appName("Analises") \
    .getOrCreate()
spark


<h2>Functions

In [3]:
def takePandas(self, n):
    rows = self.take(n)
    df = pd.DataFrame(rows, columns=self.columns)
    return df
DataFrame.takePandas = takePandas


In [4]:
def save_intermediary_step(df, path,mode="overwrite"):
        df.write.format("delta").mode(mode).save(path)

<h1>Code

In [ ]:
df_lol = spark.read.csv("..\..\datasets\League of legend Champions 2024.csv",header=True)

In [ ]:
df_lol.takePandas(10)

In [ ]:
df_lol.printSchema()

<h1> Roles distribution

Pergunta: Quais papéis (Role) são mais comuns entre os campeões?<br>
Insight: Distribuição percentual dos campeões por papel (ex.: Lutador, Mago, Suporte).<br>
Benefício: Identificar qual categoria de campeões é mais dominante e se há algum desequilíbrio entre os papéis.

In [ ]:
df_lol.count()

In [9]:
roles_distribution = (df_lol.withColumnRenamed("Classes","classes")
                      .groupBy("classes")
                      .count()
                      .withColumn("share",F.col("count")/df_lol.count())
                      .withColumn("classes",F.when(
                                                    F.col("count")<=1,F.lit("Others")
                                                  ).otherwise(F.col("classes"))
                                  )
                      .groupBy("classes","count","share")
                      .agg(
                          F.sum(F.col("count")).alias("count_"),
                          F.sum(F.col("share")).alias("share_")
                      )
                      .orderBy(F.desc("count_"))
                      .drop("count","share")
                      )

pd_roles_distribution = roles_distribution.toPandas()

In [ ]:
roles_distribution.takePandas(1)

In [ ]:
fig, ax1 = plt.subplots(1,2,figsize=(18, 5))

ax1[0].pie(pd_roles_distribution['share_'],labels = pd_roles_distribution['classes'], )
bars = ax1[1].bar(pd_roles_distribution['classes'],pd_roles_distribution['count_'],label="classes count")

for bar in bars:
    yval = bar.get_height()
    ax1[1].text(bar.get_x() + bar.get_width()/2, yval, int(yval), ha='center', va='bottom')

    
ax1[0].set_title('Classes share')
ax1[1].set_title('Classes count')



plt.xticks(rotation=90)
plt.tight_layout()


In [ ]:
#just to exemplify ETL , extraction , transform ,  and now ... load 
save_intermediary_step(roles_distribution,path="..\datasets")